In [2]:
import tensorflow as tf
import numpy as np
tf.reset_default_graph()
sentences = [ "i like dog", "i love coffee", "i hate milk"]
word_list = " ".join(sentences).split()

In [3]:
word_list = list(set(word_list))

word_dict = {w:i for i,w in enumerate(word_list)}

number_dict = {i:w for i,w in enumerate(word_list)}

n_class = len(word_dict)

n_step = 2
n_hidden = 2

In [38]:
def make_batch(sentences):
    input_batch = []
    target_batch = []
    
    for sen in sentences:
        word = sen.split()
        input = [word_dict[n] for n in word[:-1]]
        target = word_dict[word[-1]]
        #print(input)
        input_batch.append(np.eye(n_class)[input])
        target_batch.append(np.eye(n_class)[target])
    return input_batch, target_batch

In [11]:
#model
X = tf.placeholder(tf.float32, [None, n_step, n_class]) #[batch_size, number of steps, number of vocabulary]
Y = tf.placeholder(tf.float32, [None, n_class])

In [13]:
input = tf.reshape(X, shape=[-1, n_step*n_class]) # [batch_size, n_step * n_class]

In [15]:
H = tf.Variable(tf.random_normal([n_step * n_class, n_hidden]))
d = tf.Variable(tf.random_normal([n_hidden]))
U = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

In [16]:
tanh = tf.nn.tanh(d+tf.matmul(input, H))
model = tf.matmul(tanh, U)+b

In [23]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))

In [24]:
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)
prediction = tf.argmax(model,1)

In [25]:
#training
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [39]:
input_batch, target_batch = make_batch(sentences)

In [45]:
for epoch in range(5000):
    _, loss = sess.run([optimizer, cost], feed_dict={X: input_batch, Y: target_batch})
    if (epoch+1) %500 ==0:
        print("Epoch: {:4d}, cost={:.6f}".format(epoch+1, loss))

Epoch:  500, cost=1.065756
Epoch: 1000, cost=0.595331
Epoch: 1500, cost=0.262833
Epoch: 2000, cost=0.104408
Epoch: 2500, cost=0.054236
Epoch: 3000, cost=0.032912
Epoch: 3500, cost=0.021720
Epoch: 4000, cost=0.015085
Epoch: 4500, cost=0.010833
Epoch: 5000, cost=0.007962


In [46]:
predict = sess.run([prediction], feed_dict={X:input_batch})

In [54]:
input = [sen.split()[:2] for sen in sentences]
print(input,'->', [number_dict[n] for n in predict[0]])

[['i', 'like'], ['i', 'love'], ['i', 'hate']] -> ['dog', 'coffee', 'milk']
